In [46]:
import nest
import csv
import pickle
import numpy as np
import pandas as pd
import os
from collections import defaultdict
import matplotlib.pyplot as plt
nest.ResetKernel()

In [47]:
class EINetwork:
    def __init__(self) -> None:
        self.unique_id = []
        self.neurons = {}
        self.connections = defaultdict(list)
        self.num_neurons = 0
        
        # count of inh and exh synapse
        self.nI = 0
        self.nE = 0
        
        self.neuron_model = 'iaf_psc_alpha'
        
        nest.CopyModel('stdp_synapse_hom', 'e_syn')
        nest.CopyModel('static_synapse', 'i_syn')
    
    def process_and_cache_data_csv(
        self,
        csv_file_path='../connectome_materials/connections_no_threshold.csv',
        cache_file_path='../cache_file/csv_read_to_cache.pkl'
    ):
        if os.path.exists(cache_file_path):
            print('\n*** Loading connections data from cache... ***')
            with open(cache_file_path, 'rb') as f:
                self.unique_id, self.connections = pickle.load(f)
                return self.unique_id, self.connections
            
        print('\n*** Processing CSV file... ***')
        
        data = pd.read_csv(csv_file_path)
        self.unique_id = list(set(data['pre_root_id']).union(set(data['post_root_id'])))
        print('\n*** Unique_id Completed ***')
        
        def convert_nt_type(nt_type):
            return 'i_syn' if nt_type == 'GABA' else 'e_syn'
            
        grouped = data.groupby('pre_root_id')
        for pre_id, group in grouped:
            self.connections[pre_id] = group[['post_root_id', 'syn_count', 'nt_type']].values.tolist()
            self.connections[pre_id] = [
                [post_id, syn_count, convert_nt_type(nt_type)]
                for post_id, syn_count, nt_type in self.connections[pre_id]
            ]
        
        print('\n*** Saving processed data to cache... ***')
        with open(cache_file_path, 'wb') as f:
            pickle.dump((self.unique_id, self.connections), f)
        
        return self.unique_id, self.connections
            
    
    def Create_and_Connect(self):
        print('\n*** Creating Neurons... ***')
        all_neurons = nest.Create(self.neuron_model, n=len(self.unique_id))
        
        print(f'\n*** {len(self.unique_id)} neurons have been created ***')
        
        print(f'\n*** Building neurons dictionary... ***')
        self.neurons = dict(zip(self.unique_id, all_neurons))
        
        print(f'\n*** Dictionary has been created ***')
        
        print(f'\n*** Creating Connections ***')
        
        for pre_id, group in self.connections.items():
            
            pre_neuron = self.neurons[pre_id]
            
            group_array = np.array(group)
            
            post_ids = group_array[:, 0]
            syn_counts = group_array[:, 1].astype(int)
            nt_types = group_array[:, 2]
            
            print(post_ids)
            break

        
        
        
        

In [48]:
MyNETwork = EINetwork()
unique_id, connections = MyNETwork.process_and_cache_data_csv()
MyNETwork.Create_and_Connect()




*** Loading connections data from cache... ***

*** Creating Neurons... ***

*** 138639 neurons have been created ***

*** Building neurons dictionary... ***

*** Dictionary has been created ***

*** Creating Connections ***
['720575940609975854' '720575940613059993' '720575940640021838'
 '720575940619680933' '720575940623935588' '720575940624014524'
 '720575940622115169' '720575940634848875' '720575940639449459'
 '720575940618477947' '720575940606217138' '720575940608552405'
 '720575940621602081' '720575940638313449' '720575940623087507'
 '720575940621780812' '720575940626718739' '720575940629211500'
 '720575940621325993' '720575940630382966' '720575940631239491'
 '720575940632195532' '720575940616986553' '720575940615006119'
 '720575940616878370' '720575940632824095' '720575940623649560'
 '720575940635062196' '720575940620124326' '720575940630993423'
 '720575940640902389' '720575940626630693' '720575940636795620'
 '720575940632526440' '720575940645723428' '720575940621418573'
 '7205

In [49]:
for key in MyNETwork.connections.keys():
    print(MyNETwork.connections[key])
    break

[[720575940609975854, 4, 'e_syn'], [720575940613059993, 5, 'e_syn'], [720575940640021838, 1, 'e_syn'], [720575940619680933, 2, 'e_syn'], [720575940623935588, 4, 'e_syn'], [720575940624014524, 1, 'e_syn'], [720575940622115169, 5, 'e_syn'], [720575940634848875, 17, 'e_syn'], [720575940639449459, 1, 'e_syn'], [720575940618477947, 1, 'e_syn'], [720575940606217138, 1, 'e_syn'], [720575940608552405, 5, 'e_syn'], [720575940621602081, 1, 'e_syn'], [720575940638313449, 1, 'e_syn'], [720575940623087507, 1, 'e_syn'], [720575940621780812, 1, 'e_syn'], [720575940626718739, 5, 'e_syn'], [720575940629211500, 3, 'e_syn'], [720575940621325993, 2, 'e_syn'], [720575940630382966, 1, 'i_syn'], [720575940631239491, 1, 'e_syn'], [720575940632195532, 2, 'e_syn'], [720575940616986553, 5, 'e_syn'], [720575940615006119, 1, 'e_syn'], [720575940616878370, 2, 'e_syn'], [720575940632824095, 1, 'e_syn'], [720575940623649560, 2, 'e_syn'], [720575940635062196, 1, 'e_syn'], [720575940620124326, 8, 'e_syn'], [72057594063

In [50]:
print(len(connections.keys()))

138005


In [51]:
print(len(unique_id))

138639
